In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA fraud_detection")


DataFrame[]

In [0]:
from pyspark.sql.functions import col

silver_customer_df = (
    spark.table("bronze_customers")
    .select("customer_id", "country_code")
    .dropna(subset=["customer_id"])
    .dropDuplicates(["customer_id"])
)


In [0]:
(
    silver_customer_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_dim_customer")
)


In [0]:
%sql
SELECT COUNT(*) FROM silver_dim_customer;
SELECT * FROM silver_dim_customer LIMIT 5;


customer_id,country_code
C2317870228,TUR
C4603372195,REU
C1189806313,CAN
C9679880050,ESP
C2796449758,ESP


In [0]:
silver_country_df = (
    spark.table("bronze_country_code")
    .select("country_code")
    .dropna()
    .dropDuplicates()
)


In [0]:
(
    silver_country_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_dim_country")
)


In [0]:
%sql
SELECT COUNT(*) FROM silver_dim_country;
SELECT * FROM silver_dim_country;


country_code
CAN
ITA
ESP
QAT
PSE
PER
TUR
RUS
FRA
KHM


In [0]:
#
#
#------------silver_fact_transactions-----------------
#
#

In [0]:
from pyspark.sql.functions import col

silver_txn_df = (
    spark.table("bronze_transactions")
    .select(
        col("id").alias("transaction_id"),
        "step",
        "type",
        col("amount").cast("double"),
        col("nameOrig").alias("customer_id"),
        col("nameDest").alias("merchant_id"),
        col("oldBalanceOrig").cast("double"),
        col("newBalanceOrig").cast("double"),
        col("oldBalanceDest").cast("double"),
        col("newBalanceDest").cast("double"),
        "countryOrig",
        "countryDest",
        "ingest_time"
    )
)


In [0]:
(
    silver_txn_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_fact_transactions")
)


In [0]:
%sql
SELECT COUNT(*) FROM silver_fact_transactions;
SELECT * FROM silver_fact_transactions LIMIT 5;


transaction_id,step,type,amount,customer_id,merchant_id,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,countryOrig,countryDest,ingest_time
94a0eaf0-3b62-460c-93c6-72e40bb11974,1,CASH_IN,78994.9,C5669199819,M6571322302,130484.07,209478.97,0.0,0.0,FRA,ITA,2026-02-08T21:37:25.820Z
6aa1b852-8aee-4e7c-af41-53561512a79d,1,CASH_IN,130967.02,C0165182576,M8239546933,35.88,131002.89,0.0,0.0,REU,PER,2026-02-08T21:37:25.820Z
1380fad7-5575-4f23-81d7-c7cab1c03458,1,CASH_IN,172473.56,C4621556856,M1469655819,52.79,172526.35,8022.82,8022.82,NGA,BRA,2026-02-08T21:37:25.820Z
2600ef3a-af26-410f-b97f-41bbbecbff4d,1,CASH_IN,284398.59,C7349398929,M1719835988,134044.23,418442.83,10058.01,10058.01,ESP,TUR,2026-02-08T21:37:25.820Z
5ed03836-07a9-4cc6-b332-a08225b3dada,2,CASH_IN,42972.58,C9441668199,M6087015760,447311.47,490284.05,0.0,0.0,FRA,CAN,2026-02-08T21:37:25.820Z


In [0]:
#
#
#-------Silver_fact_fraud-----------
#
#


In [0]:
silver_fraud_df = (
    spark.table("bronze_fraud_report")
    .select(
        col("id").alias("transaction_id"),
        col("fraud_flag").cast("int")
    )
)


In [0]:
(
    silver_fraud_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_fact_fraud")
)


In [0]:
%sql
SELECT fraud_flag, COUNT(*) 
FROM silver_fact_fraud
GROUP BY fraud_flag;


fraud_flag,COUNT(*)
1,12380
0,5430616


In [0]:
%sql
SHOW TABLES;


database,tableName,isTemporary
fraud_detection,bronze_country_code,false
fraud_detection,bronze_customers,false
fraud_detection,bronze_fraud_report,false
fraud_detection,bronze_transactions,false
fraud_detection,silver_dim_country,false
fraud_detection,silver_dim_customer,false
fraud_detection,silver_fact_fraud,false
fraud_detection,silver_fact_transactions,false


In [0]:
#-------------------------------------------
#
#---------Silver_Normalization--------------
#
#-------------------------------------------

In [0]:
silver_dim_customer_df = (
    spark.table("bronze_customers")
    .select(
        "customer_id",
        "country_code"
    )
    .dropna(subset=["customer_id"])
    .dropDuplicates(["customer_id"])
)


In [0]:
(
    silver_dim_customer_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_dim_customer")
)


In [0]:
%sql
SELECT COUNT(*) FROM silver_dim_customer;
SELECT * FROM silver_dim_customer LIMIT 5;


customer_id,country_code
C2317870228,TUR
C4603372195,REU
C1189806313,CAN
C9679880050,ESP
C2796449758,ESP


In [0]:
silver_dim_country_df = (
    spark.table("bronze_country_code")
    .select("country_code")
    .dropna()
    .dropDuplicates()
)


In [0]:
(
    silver_dim_country_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_dim_country")
)


In [0]:
%sql
SELECT * FROM silver_dim_country;


country_code
CAN
ITA
ESP
QAT
PSE
PER
TUR
RUS
FRA
KHM


In [0]:
from pyspark.sql.functions import col

silver_fact_transactions_df = (
    spark.table("bronze_transactions")
    .select(
        col("id").alias("transaction_id"),
        col("step").cast("int"),
        col("type"),
        col("amount").cast("double"),
        col("nameOrig").alias("customer_id"),
        col("nameDest").alias("merchant_id"),
        col("oldBalanceOrig").cast("double"),
        col("newBalanceOrig").cast("double"),
        col("oldBalanceDest").cast("double"),
        col("newBalanceDest").cast("double"),
        col("countryOrig"),
        col("countryDest"),
        col("ingest_time")
    )
)


In [0]:
(
    silver_fact_transactions_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("silver_fact_transactions")
)


In [0]:
%sql
DROP TABLE IF EXISTS silver_fact_transactions;


In [0]:
%sql
DESCRIBE silver_fact_transactions;
SELECT COUNT(*) FROM silver_fact_transactions;
SELECT * FROM silver_fact_transactions LIMIT 5;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8907920231679062>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'DESCRIBE silver_fact_transactions;\nSELECT COUNT(*) FROM silver_fact_transactions;\nSELECT * FROM silver_fact_transactions LIMIT 5;\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:194, in SqlMagic.sql

In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA fraud_detection")


DataFrame[]

In [0]:
%sql
SELECT current_catalog(), current_schema();


current_catalog(),current_schema()
workspace,fraud_detection


In [0]:
from pyspark.sql.functions import col

silver_fact_transactions_df = (
    spark.table("bronze_transactions")
    .select(
        col("id").alias("transaction_id"),
        col("step").cast("int"),
        col("type").alias("transaction_type"),
        col("amount").cast("double"),
        col("nameOrig").alias("origin_account"),
        col("nameDest").alias("destination_account"),
        col("oldBalanceOrig").cast("double"),
        col("newBalanceOrig").cast("double"),
        col("oldBalanceDest").cast("double"),
        col("newBalanceDest").cast("double"),
        col("isUnauthorizedOverdraft").cast("int"),
        col("ingest_time")
    )
)


In [0]:
(
    silver_fact_transactions_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("silver_fact_transactions")
)


In [0]:
%sql
DESCRIBE silver_fact_transactions;

SELECT COUNT(*) FROM silver_fact_transactions;

SELECT *
FROM silver_fact_transactions
LIMIT 5;


transaction_id,step,transaction_type,amount,origin_account,destination_account,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,isUnauthorizedOverdraft,ingest_time
e87bd2ce-e892-4ff2-b559-1dff52a96e0b,1,CASH_IN,15504.98,C7252612212,M4639849776,183279.61,198784.59,0.0,0.0,0,2026-02-08T21:37:25.820Z
ddcec6e4-aeb5-4165-8a85-5e60304013aa,1,CASH_IN,83938.99,C7567192866,M0172269636,5578.3,89517.29,0.0,0.0,0,2026-02-08T21:37:25.820Z
e13f5635-788a-4617-8bf7-7388acefe360,1,CASH_IN,131805.11,C8879445092,M2384302511,29222.15,161027.26,0.0,0.0,0,2026-02-08T21:37:25.820Z
1402e4cf-9173-4b00-bdac-a6ee620a509b,1,CASH_IN,176931.04,C7110900402,M8592581184,52851.85,229782.89,0.0,0.0,0,2026-02-08T21:37:25.820Z
9b986898-70af-4d30-9f7e-c3b2da193cc9,1,CASH_OUT,268084.67,CC1229472380,M0141939366,268084.67,0.0,0.0,0.0,0,2026-02-08T21:37:25.820Z


In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA fraud_detection")



DataFrame[]

In [0]:
from pyspark.sql.functions import col

silver_dim_customers_df = (
    spark.table("bronze_transactions")
    .select(col("nameOrig").alias("customer_id"))
    .union(
        spark.table("bronze_transactions")
        .select(col("nameDest").alias("customer_id"))
    )
    .dropna()
    .dropDuplicates(["customer_id"])
)


In [0]:
(
    silver_dim_customers_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_dim_customers")
)


In [0]:
%sql
SELECT COUNT(*) FROM silver_dim_customers;
SELECT * FROM silver_dim_customers LIMIT 5;


customer_id
C7168983926
C2401457690
C2317870228
C1189806313
CC0189065943


In [0]:
silver_dim_countries_df = (
    spark.table("bronze_transactions")
    .select(col("countryOrig").alias("country_code"))
    .union(
        spark.table("bronze_transactions")
        .select(col("countryDest").alias("country_code"))
    )
    .dropna()
    .dropDuplicates(["country_code"])
)


In [0]:
(
    silver_dim_countries_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_dim_countries")
)


In [0]:
%sql
SELECT COUNT(*) FROM silver_dim_countries;
SELECT * FROM silver_dim_countries;


country_code
CAN
ITA
ESP
QAT
PSE
PER
TUR
RUS
FRA
KHM


In [0]:
silver_dim_fraud_flags_df = (
    spark.table("bronze_transactions")
    .select(
        col("id").alias("transaction_id"),
        col("isUnauthorizedOverdraft").cast("int").alias("unauthorized_overdraft_flag")
    )
)


In [0]:
(
    silver_dim_fraud_flags_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_dim_fraud_flags")
)


In [0]:
%sql
SELECT COUNT(*) FROM silver_dim_fraud_flags;
SELECT * FROM silver_dim_fraud_flags LIMIT 5;


transaction_id,unauthorized_overdraft_flag
90c0e73f-3878-41a7-8964-609e10f2c1b4,0
5055ca4b-d16d-4a05-97c7-a9e38fb9fb85,0
34db2fa6-1c2b-4e47-919b-fb5f1015276a,0
4eba2416-4b83-477d-8aec-fc056f1ccf38,0
ef0bb47c-d9c5-4f16-8cfb-d986084e1695,0


In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA fraud_detection;


In [0]:
%sql
CREATE OR REPLACE TABLE silver_dim_accounts
AS
SELECT DISTINCT
    origin_account AS account_id
FROM silver_fact_transactions
WHERE origin_account IS NOT NULL

UNION

SELECT DISTINCT
    destination_account AS account_id
FROM silver_fact_transactions
WHERE destination_account IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM silver_dim_accounts;


COUNT(*)
89686


In [0]:
%sql
SELECT * FROM silver_dim_accounts LIMIT 10;


account_id
C2317870228
C4603372195
C1189806313
C9679880050
C2796449758
C7168983926
C2401457690
CC9915820871
CC0786317222
CC6000864049


In [0]:
%sql
DESCRIBE silver_dim_accounts;


col_name,data_type,comment
account_id,string,null


In [0]:
%sql
DESCRIBE bronze_customers;


col_name,data_type,comment
customer_id,string,null
country_code,string,null
ingest_time,timestamp,null
